In [18]:
# Tagging sentences using the default tagger
!nohup python3 tagger.py --files ../../2023-01-04/proof-corpus/sent01.tsv --cores 25 --output tagged_sents/tagged_sentences_test.txt

nohup: ignoring input and appending output to 'nohup.out'


In [19]:
# Create word bins for optimal tagger
!nohup python3 extract_sents.py --file tagged_sents/tagged_sentences_test.txt --word_file word_bin_list.txt --cores 20 --unique

nohup: ignoring input and appending output to 'nohup.out'


In [20]:
# Make testing and training bins
!nohup python3 make_test_train.py --num_train_sents 5 --num_test_bins 5 --train_extension test --test_extension test --word_list word_lists/nnp_verb_list_test.txt

nohup: ignoring input and appending output to 'nohup.out'


In [21]:
# Run main experiment
!nohup python3 main_experiment.py --train training_set/test.txt --test testing_set/test.txt --cores 30 --num_trials 5 --train_num_list 1,5 --iter_num_list 5 --extension test --wsj_test --dump --default_results 


nohup: ignoring input and appending output to 'nohup.out'


In [22]:
# Read experiment results
!python3 read_experiment_results.py --files experiments/*test* --output experiments/summary_test.txt


experiment_1sents_5iters_test-wsj
[['0.97008534755474', '4', '4', '66', '164', '17', '32', '175', '22', '147', '3130'], ['0.9698368552341085', '5', '4', '70', '166', '16', '32', '171', '22', '156', '3156'], ['0.9698655274249506', '5', '4', '64', '175', '14', '31', '169', '23', '149', '3153'], ['0.9700375605700031', '3', '4', '64', '157', '12', '30', '175', '23', '146', '3135'], ['0.9703242824784242', '4', '4', '61', '164', '16', '24', '175', '25', '147', '3105']]
experiment_1sents_5iters_test
[['0.9461538461538461', '10', '0', '0', '3', '0', '0', '3', '0', '23', '42'], ['0.9564102564102565', '10', '0', '0', '7', '0', '1', '1', '0', '17', '34'], ['0.958974358974359', '9', '0', '0', '2', '0', '0', '2', '0', '16', '32'], ['0.9525641025641025', '12', '0', '0', '4', '0', '0', '3', '0', '20', '37'], ['0.9346153846153846', '16', '0', '0', '2', '0', '0', '4', '0', '30', '51']]
experiment_5sents_5iters_test-wsj
[['0.970372069463161', '4', '4', '63', '166', '14', '33', '177', '24', '141', '3100'

In [23]:
# Make new testing bin based on training bin
!nohup python3 make_test_train.py --train training_set/test.txt --num_test_bins 1 --test_extension test_2 --word_list word_lists/nnp_verb_list_partition.txt


nohup: ignoring input and appending output to 'nohup.out'


In [24]:
# Run dumped experiment
!nohup python3 dumped_main_experiment.py --tagger tagger/5sents_5iters_test_trial1.pk --test testing_set/refer_handtagged.txt --extension refer_test -p 25 --tag_n 3

nohup: ignoring input and appending output to 'nohup.out'


In [26]:
# Compare tagger weights in 7_5/5sents_5iters_7_5_trial
!python3 compare_tagger_weights.py --file disagreeing_sents/disagreeing_sents_7_8.txt --output compare_weights_test.txt

loaded
